In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [3]:
claim_data = pd.read_csv('claim_data_group5_2024.csv')
claim_data.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,2271893.0,0.0,0.83,E,5.0,17.0,53.0,64.0,B2,Diesel,3317.0,R93,0.0
1,1111864.0,0.0,0.24,E,5.0,2.0,27.0,64.0,B3,Diesel,2740.0,R22,0.0
2,72908.0,0.0,0.50,E,7.0,11.0,67.0,50.0,B3,Regular,4762.0,R93,0.0
3,2283027.0,0.0,0.08,B,5.0,8.0,28.0,60.0,B1,Diesel,64.0,R91,0.0
4,1123838.0,0.0,0.03,A,11.0,1.0,38.0,50.0,B2,Regular,16.0,R24,0.0


In [4]:
# Define driver age categories and vehicle power levels as risk group identifiers
claim_data['DrivAgeGroup'] = pd.cut(claim_data['DrivAge'], bins=[18, 25, 35, 50, 65, 100], labels=['18-25', '26-35', '36-50', '51-65', '65+'])
claim_data['RiskGroup'] = claim_data['DrivAgeGroup'].astype(str) + "_" + claim_data['VehPower'].astype(str)


In [5]:
# Calculate claim frequency and severity per risk group
risk_group_stats = claim_data.groupby('RiskGroup').agg(
    frequency=('ClaimNb', 'mean'),  # Average claims per policy
    severity=('ClaimAmount', lambda x: x[x > 0].mean())  # Average claim amount for policies with claims
).fillna(0)  # Replace NaN values with 0 for groups with no claims

risk_group_stats.reset_index(inplace=True)
risk_group_stats.head()


,RiskGroup,frequency,severity
0,18-25_10.0,0.036036,718.7925
1,18-25_11.0,0.122449,1235.8700
2,18-25_12.0,0.000000,0.0000
3,18-25_13.0,0.000000,0.0000
4,18-25_14.0,0.187500,746.7100


In [6]:
# Calculate expected loss for each risk group
risk_group_stats['expected_loss'] = risk_group_stats['frequency'] * risk_group_stats['severity']
risk_group_stats.head()


,RiskGroup,frequency,severity,expected_loss
0,18-25_10.0,0.036036,718.7925,25.902432
1,18-25_11.0,0.122449,1235.8700,151.331020
2,18-25_12.0,0.000000,0.0000,0.000000
3,18-25_13.0,0.000000,0.0000,0.000000
4,18-25_14.0,0.187500,746.7100,140.008125


In [7]:
# Assume a buffer multiplier based on 99.5% confidence using the normal distribution (z-score ~2.576)
z_score_99_5 = norm.ppf(0.995)

# Estimate premium as expected loss plus buffer
risk_group_stats['premium'] = risk_group_stats['expected_loss'] * (1 + z_score_99_5)
risk_group_stats[['RiskGroup', 'frequency', 'severity', 'expected_loss', 'premium']]


,RiskGroup,frequency,severity,expected_loss,premium
0,18-25_10.0,0.036036,718.7925,25.902432,92.622677
1,18-25_11.0,0.122449,1235.8700,151.331020,541.133897
2,18-25_12.0,0.000000,0.0000,0.000000,0.000000
3,18-25_13.0,0.000000,0.0000,0.000000,0.000000
4,18-25_14.0,0.187500,746.7100,140.008125,500.645156
...,...,...,...,...,...
64,nan_5.0,0.071429,3878.8050,277.057500,990.710327
65,nan_6.0,0.000000,0.0000,0.000000,0.000000
66,nan_7.0,0.130435,2087.5300,272.286522,973.650123
67,nan_8.0,0.000000,0.0000,0.000000,0.000000


In [8]:
# Display the final risk group statistics with calculated premiums
print(risk_group_stats[['RiskGroup', 'frequency', 'severity', 'expected_loss', 'premium']])


     RiskGroup  frequency   severity  expected_loss     premium
0   18-25_10.0   0.036036   718.7925      25.902432   92.622677
1   18-25_11.0   0.122449  1235.8700     151.331020  541.133897
2   18-25_12.0   0.000000     0.0000       0.000000    0.000000
3   18-25_13.0   0.000000     0.0000       0.000000    0.000000
4   18-25_14.0   0.187500   746.7100     140.008125  500.645156
..         ...        ...        ...            ...         ...
64     nan_5.0   0.071429  3878.8050     277.057500  990.710327
65     nan_6.0   0.000000     0.0000       0.000000    0.000000
66     nan_7.0   0.130435  2087.5300     272.286522  973.650123
67     nan_8.0   0.000000     0.0000       0.000000    0.000000
68     nan_9.0   0.000000     0.0000       0.000000    0.000000

[69 rows x 5 columns]
